<a href="https://colab.research.google.com/github/Ordo-Umbra/Recursive-Complexity-Model/blob/main/RCM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install networkx

In [ ]:
import numpy as np
import networkx as nx
import json
import os
from sklearn.metrics.pairwise import cosine_similarity

class RCMAgent:
    def __init__(self, d=128, kappa=1.0, lambda_=0.1, theta=0.2, epsilon=1e-3, seed=42, decay_gamma=0.99, stability_sigma=0.8, prune_tau=-0.2, holo_dim=64, use_holo=True):
        self.d = d
        self.kappa = kappa
        self.lambda_ = lambda_
        self.theta = theta
        self.epsilon = epsilon
        self.seed = seed
        self.rng = np.random.RandomState(seed)
        self.decay_gamma = decay_gamma
        self.stability_sigma = stability_sigma
        self.prune_tau = prune_tau
        self.holo_dim = holo_dim
        self.use_holo = use_holo
        self.atom_labels = [f'Atom{i}' for i in range(d)]
        self.trace = []
        self.stable_symbols = {}
        self.symbol_id = 0
        self.state_file = 'rcm_state.json'
        self.holo_proj = self._init_hologram() if use_holo else None
        self.stagnation_count = 0
        self.stagnation_threshold = 0.05
        self.occupancy_threshold = 0.8  # % active for expansion trigger
        self.session_id = 1
        self.load_state()
        if len(self.trace) == 0:
            self.reset_state()
            print(f"Starting fresh session (d={self.d}, holo_dim={self.holo_dim}, use_holo={use_holo}).")

    def _init_hologram(self):
        # Random features for node embeddings: (d, holo_dim)
        return self.rng.normal(0, 1/np.sqrt(self.holo_dim), (self.d, self.holo_dim))

    def reset_state(self):
        self.v = np.full(self.d, 0.5, dtype=np.float32)
        self.G = nx.Graph()
        self.G.add_nodes_from(range(self.d))
        self.stable_symbols = {}
        self.symbol_id = 0
        self.stagnation_count = 0
        self.session_id = 1
        if self.use_holo:
            self.holo_proj = self._init_hologram()
        self.update_metrics()

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

    def logit(self, p):
        p = np.clip(p, 1e-8, 1 - 1e-8)
        return np.log(p / (1 - p))

    def entropy(self, v):
        p = np.clip(v, 1e-10, 1 - 1e-10)
        q = 1 - p
        h = np.zeros_like(p)
        mask = (p > 0) & (p < 1)
        h[mask] = - (p[mask] * np.log2(p[mask]) + q[mask] * np.log2(q[mask]))
        return np.sum(h)

    def project_to_holo(self, v):
        # Node-wise: (d,) * (d, holo_dim) -> (d, holo_dim) embeddings
        return self.holo_proj * v[:, np.newaxis]

    def update_metrics(self):
        self.current_c = self.entropy(self.v) + self.lambda_ * self.G.number_of_edges()
        if self.use_holo:
            v_holo = self.project_to_holo(self.v)
            self.current_i = self.compute_coherence_holo(v_holo, self.G)
        else:
            self.current_i = self.compute_coherence_full(self.v, self.G)

    def compute_coherence_holo(self, v_holo, g_temp):
        total_i = 0
        num_edges = 0
        for i, j in g_temp.edges():
            diff_norm = np.linalg.norm(v_holo[i] - v_holo[j])
            total_i += 1 - (diff_norm / np.sqrt(2 * self.holo_dim))  # Normalized dist [0,1]
            num_edges += 1
        return total_i / max(1, num_edges)

    def compute_coherence_full(self, v_temp, g_temp):
        total_i = 0
        num_edges = 0
        for i, j in g_temp.edges():
            total_i += 1 - abs(v_temp[i] - v_temp[j])
            num_edges += 1
        return total_i / max(1, num_edges)

    def modulated_decay(self):
        scores = np.array([self.v[i] * self.G.degree[i] for i in range(self.d)])
        gammas = self.decay_gamma * (1 - np.clip(scores / self.stability_sigma, 0, 1))
        self.v *= gammas
        self.v = np.clip(self.v, 0, 1)
        low_v_nodes = np.where(self.v < self.theta)[0]
        for node in low_v_nodes:
            if not self.is_stable_node(node):
                self.G.remove_edges_from(list(self.G.edges(node)))
                self.v[node] = 0
        self.update_metrics()

    def is_stable_node(self, node):
        return any(node in nodes for nodes in self.stable_symbols.values())

    def check_and_prune_conflict(self, pre_s):
        post_s = self.current_c + self.kappa * self.current_i
        delta_conflict = post_s - pre_s
        if delta_conflict < self.prune_tau:
            print(f"Conflict detected (ΔS={delta_conflict:.3f} < {self.prune_tau}). Pruning weak stables...")
            stable_scores = {sid: np.mean([self.v[n] for n in nodes]) for sid, nodes in self.stable_symbols.items()}
            if stable_scores:
                weakest_sid = min(stable_scores, key=stable_scores.get)
                for node in self.stable_symbols[weakest_sid]:
                    self.G.remove_edges_from(list(self.G.edges(node)))
                del self.stable_symbols[weakest_sid]
                print(f"Pruned stable: {weakest_sid}")

    def generate_actions(self, num_actions=6):
        active_dims = np.where(self.v > self.theta)[0]
        if len(active_dims) == 0:
            active_dims = np.arange(self.d)[:min(10, self.d)]
        actions = []
        delta = 0.1
        i = self.rng.choice(active_dims)
        actions.append(lambda v, g: v.__setitem__(i, np.clip(v[i] + delta, 0, 1)))
        actions.append(lambda v, g: v.__setitem__(i, np.clip(v[i] - delta, 0, 1)))
        j = self.rng.choice(active_dims)
        has_edge = self.G.has_edge(i, j)
        actions.append(lambda v, g: self.remove_edge_from_temp(g, i, j) if has_edge else self.add_edge_from_temp(g, i, j))
        actions.append(lambda v, g: self.add_edge_from_temp(g, i, j) if has_edge else self.remove_edge_from_temp(g, i, j))
        top_beliefs_list = self.top_beliefs(2)
        top_idx = [b['idx'] for b in top_beliefs_list]
        if len(top_idx) == 2:
            actions.append(lambda v, g: self.merge_top(v, top_idx))
        else:
            actions.append(lambda v, g: None)
        actions.append(lambda v, g: self.add_noise(v, active_dims))
        return actions[:num_actions]

    def add_edge_from_temp(self, g_temp, i, j):
        if i != j and not g_temp.has_edge(i, j):
            g_temp.add_edge(i, j)

    def remove_edge_from_temp(self, g_temp, i, j):
        if g_temp.has_edge(i, j):
            g_temp.remove_edge(i, j)

    def merge_top(self, v, top_idx):
        avg = (v[top_idx[0]] + v[top_idx[1]]) / 2
        v[top_idx[0]] = avg
        v[top_idx[1]] = avg

    def add_noise(self, v, active_dims):
        noise = self.rng.uniform(-0.05, 0.05, len(active_dims))
        v[active_dims] += noise
        v[active_dims] = np.clip(v[active_dims], 0, 1)

    def stabilize_symbols(self, best_s):
        if best_s > self.stability_sigma:
            scores = [(self.v[i] * self.G.degree[i], i) for i in range(self.d)]
            scores.sort(reverse=True)
            high_nodes = [idx for _, idx in scores[:3]]
            if not any(set(high_nodes).issubset(set(nodes)) for nodes in self.stable_symbols.values()):
                sid = f'Symbol{self.symbol_id}'
                self.stable_symbols[sid] = high_nodes
                self.symbol_id += 1
                print(f"Stabilized new symbol: {sid} -> {high_nodes}")

    def auto_expand(self, recent_s_list):
        mean_s = np.mean(recent_s_list[-5:]) if len(recent_s_list) >= 5 else np.mean(recent_s_list)
        active_ratio = len(np.where(self.v > self.theta)[0]) / self.d
        if mean_s < self.stagnation_threshold or active_ratio > self.occupancy_threshold:
            self.stagnation_count += 1
            if self.stagnation_count >= 3:
                old_d = self.d
                expand_size = 64
                self.d += expand_size
                new_v = np.full(expand_size, 0.5, dtype=np.float32)
                self.v = np.append(self.v, new_v)
                self.G.add_nodes_from(range(old_d, self.d))
                # Sparse random edges in new space
                for _ in range(5):
                    i = self.rng.randint(old_d, self.d)
                    j = self.rng.randint(old_d, self.d)
                    if i != j:
                        self.G.add_edge(i, j)
                self.atom_labels.extend([f'Atom{i}' for i in range(old_d, self.d)])
                if self.use_holo:
                    self.holo_proj = self._init_hologram()
                self.stagnation_count = 0
                print(f"System expanded: d={old_d} → {self.d} (S={mean_s:.3f}, occupancy={active_ratio:.2f})")

    def step(self, query=None):
        pre_s = self.current_c + self.kappa * self.current_i
        if query:
            self.inject_observation(query)
            self.check_and_prune_conflict(pre_s)
        self.modulated_decay()
        self.update_metrics()
        best_s = -np.inf
        best_v = self.v.copy()
        best_g = self.G.copy()
        best_delta_c = 0
        best_delta_i = 0
        actions = self.generate_actions()

        for act in actions:
            v_temp = self.v.copy()
            g_temp = self.G.copy()
            if act:
                act(v_temp, g_temp)

            v_temp[v_temp < self.theta] = 0
            v_temp[v_temp > 1 - self.theta] = 1

            temp_c = self.entropy(v_temp) + self.lambda_ * g_temp.number_of_edges()
            if self.use_holo:
                v_holo_temp = self.project_to_holo(v_temp)
                temp_i = self.compute_coherence_holo(v_holo_temp, g_temp)
            else:
                temp_i = self.compute_coherence_full(v_temp, g_temp)
            delta_c = temp_c - self.current_c
            delta_i = temp_i - self.current_i
            s = delta_c + self.kappa * delta_i

            if s > best_s:
                best_s = s
                best_v = v_temp
                best_g = g_temp
                best_delta_c = delta_c
                best_delta_i = delta_i

        if best_g.number_of_edges() < 1:
            i = self.rng.randint(0, self.d)
            j = self.rng.randint(0, self.d)
            if i != j:
                self.add_edge_from_temp(best_g, i, j)

        self.v = best_v
        self.G = best_g
        self.update_metrics()

        self.stabilize_symbols(best_s)

        beta = 0.01
        self.kappa = np.clip(self.kappa + beta * (1 if best_s > 0 else -1) * abs(best_delta_i), 0.1, 10)

        self.trace.append({'t': len(self.trace) + 1, 'S': float(best_s), 'deltaC': float(best_delta_c), 'deltaI': float(best_delta_i), 'kappa': float(self.kappa)})

        return {'conclusion': self.top_beliefs(3), 'trace': self.trace[-1], 'fullTrace': self.trace}

    def inject_observation(self, query):
        if not query:
            return
        words = [w for w in query.lower().split() if len(w) > 1]
        boosts = np.zeros(self.d)
        for word in words:
            idx = ord(word[0]) % self.d
            boosts[idx] += 0.2
        for i in range(self.d):
            self.v[i] = self.sigmoid(self.logit(self.v[i]) + boosts[i])
        boosted = np.where(boosts > 0)[0]
        for k in range(len(boosted)):
            for m in range(k + 1, len(boosted)):
                self.G.add_edge(boosted[k], boosted[m])
        # Link to nearest stable (1D sim)
        for boosted_node in boosted:
            boosted_v = self.v[boosted_node]
            best_sim = -1
            best_symbol = None
            for sid, nodes in self.stable_symbols.items():
                symbol_mean = np.mean(self.v[nodes])
                sim = 1 - abs(boosted_v - symbol_mean)
                if sim > best_sim:
                    best_sim = sim
                    best_symbol = sid
            if best_symbol and best_sim > 0.5:
                for node in self.stable_symbols[best_symbol]:
                    self.G.add_edge(boosted_node, node)
        self.update_metrics()

    def top_beliefs(self, m):
        scores = []
        for i in range(self.d):
            neighbors = self.G.degree[i]
            score = self.v[i] * neighbors
            scores.append((score, i))
        scores.sort(reverse=True)
        return [{'idx': idx, 'label': self.atom_labels[idx], 'confidence': f'{self.v[idx]:.2f}', 'neighbors': self.G.degree[idx]} for _, idx in scores[:m]]

    def run(self, steps, query=None, verbose=False):
        recent_s = []
        if query:
            self.inject_observation(query)
        for t in range(steps):
            out = self.step()
            recent_s.append(out['trace']['S'])
            if verbose:
                print(f"Step {t+1}: S={out['trace']['S']:.4f}, κ={out['trace']['kappa']:.2f}, Top: {out['conclusion']}")
            if abs(out['trace']['S']) < self.epsilon:
                break
            self.auto_expand(recent_s)
        return {'finalConclusion': self.top_beliefs(5), 'fullTrace': self.trace, 'state': {'v': self.v.tolist(), 'edges': list(self.G.edges())}}

    def save_state(self):
        state = {
            'v': self.v.tolist(),
            'edges': [(int(e[0]), int(e[1])) for e in self.G.edges()],
            'kappa': float(self.kappa),
            'trace': self.trace,
            'stable_symbols': {k: [int(n) for n in v] for k, v in self.stable_symbols.items()},
            'symbol_id': self.symbol_id,
            'session_id': self.session_id,
            'd': self.d,
            'holo_dim': self.holo_dim,
            'use_holo': self.use_holo
        }
        if self.use_holo:
            state['holo_proj'] = self.holo_proj.tolist()
        with open(self.state_file, 'w') as f:
            json.dump(state, f)
        print(f"State saved (d={self.d}). Stable symbols: {len(state['stable_symbols'])}")

    def load_state(self):
        try:
            with open(self.state_file, 'r') as f:
                state = json.load(f)
            self.d = state['d']
            self.v = np.array(state['v'], dtype=np.float32)
            self.G = nx.Graph()
            self.G.add_nodes_from(range(self.d))
            self.G.add_edges_from(state['edges'])
            self.kappa = state['kappa']
            self.trace = state['trace']
            self.stable_symbols = {k: state['stable_symbols'][k] for k in state['stable_symbols']}
            self.symbol_id = state.get('symbol_id', 0)
            self.holo_dim = state['holo_dim']
            self.use_holo = state.get('use_holo', True)
            if self.use_holo:
                self.holo_proj = np.array(state['holo_proj'])
            self.session_id = state.get('session_id', 0) + 1
            self.atom_labels = [f'Atom{i}' for i in range(self.d)]
            self.update_metrics()
            print(f"Resumed (d={self.d}). Stable symbols: {len(self.stable_symbols)}")
        except FileNotFoundError:
            print("New session.")

    def get_status(self):
        print(f"d={self.d}, Active stables: {self.stable_symbols}")
        print(f"Top beliefs: {self.top_beliefs(3)}")
        print(f"Edges: {len(self.G.edges())}")

# Test Function
def test_feed(agent, query, steps=3, verbose=True):
    print(f"\n--- Feeding: '{query}' ---")
    result = agent.run(steps, query, verbose)
    print("\nTop Beliefs:")
    for b in result['finalConclusion'][:3]:
        print(f"- {b['label']} (conf: {b['confidence']}, neighbors: {b['neighbors']})")
    if result['fullTrace']:
        last = result['fullTrace'][-1]
        print(f"Last S: {last['S']:.4f}")
    print(f"Edges: {len(result['state']['edges'])}")
    print(f"Stable Symbols: {len(agent.stable_symbols)}")
    agent.save_state()
    return result

# Run Tests
agent = RCMAgent(d=128)
test_feed(agent, "apple fruit red")
test_feed(agent, "banana yellow sweet")
test_feed(agent, "apple vegetable not-fruit")
agent.get_status()

New session.
Starting fresh session (d=128, holo_dim=64, use_holo=True).

--- Feeding: 'apple fruit red' ---
Stabilized new symbol: Symbol0 -> [52, 15, 127]
Step 1: S=1.0365, κ=1.01, Top: [{'idx': 52, 'label': 'Atom52', 'confidence': '0.50', 'neighbors': 1}, {'idx': 15, 'label': 'Atom15', 'confidence': '0.50', 'neighbors': 1}, {'idx': 127, 'label': 'Atom127', 'confidence': '0.50', 'neighbors': 0}]
Step 2: S=-1.2971, κ=1.01, Top: [{'idx': 113, 'label': 'Atom113', 'confidence': '0.49', 'neighbors': 1}, {'idx': 6, 'label': 'Atom6', 'confidence': '0.49', 'neighbors': 1}, {'idx': 127, 'label': 'Atom127', 'confidence': '0.49', 'neighbors': 0}]
Step 3: S=0.0609, κ=1.01, Top: [{'idx': 78, 'label': 'Atom78', 'confidence': '0.49', 'neighbors': 1}, {'idx': 59, 'label': 'Atom59', 'confidence': '0.49', 'neighbors': 1}, {'idx': 127, 'label': 'Atom127', 'confidence': '0.49', 'neighbors': 0}]
System expanded: d=128 → 192 (S=-0.067, occupancy=0.95)

Top Beliefs:
- Atom186 (conf: 0.50, neighbors: 1)
- A

In [ ]:
test_feed(agent, "apple ripe fruit")  # Validates fruit, should raise S, prune vegetable edges
agent.get_status()


--- Feeding: 'apple ripe fruit' ---
Step 1: S=0.0710, κ=1.01, Top: [{'idx': 180, 'label': 'Atom180', 'confidence': '0.47', 'neighbors': 1}, {'idx': 106, 'label': 'Atom106', 'confidence': '0.45', 'neighbors': 1}, {'idx': 319, 'label': 'Atom319', 'confidence': '0.50', 'neighbors': 0}]
Step 2: S=0.0668, κ=1.01, Top: [{'idx': 263, 'label': 'Atom263', 'confidence': '0.49', 'neighbors': 1}, {'idx': 9, 'label': 'Atom9', 'confidence': '0.45', 'neighbors': 1}, {'idx': 319, 'label': 'Atom319', 'confidence': '0.49', 'neighbors': 0}]
Step 3: S=0.0729, κ=1.01, Top: [{'idx': 243, 'label': 'Atom243', 'confidence': '0.47', 'neighbors': 1}, {'idx': 26, 'label': 'Atom26', 'confidence': '0.44', 'neighbors': 1}, {'idx': 319, 'label': 'Atom319', 'confidence': '0.49', 'neighbors': 0}]
System expanded: d=320 → 384 (S=0.070, occupancy=0.81)

Top Beliefs:
- Atom381 (conf: 0.50, neighbors: 1)
- Atom374 (conf: 0.50, neighbors: 1)
- Atom373 (conf: 0.50, neighbors: 1)
Last S: 0.0729
Edges: 7
Stable Symbols: 1
Sta

In [ ]:
test_feed(agent, "apple sweet fruit pie")  # Validates/strengthens fruit
# Or for conflict:
# test_feed(agent, "apple savory vegetable stew")  # Should prune if ΔS < -0.2
agent.get_status()


--- Feeding: 'apple sweet fruit pie' ---
Step 1: S=0.0710, κ=1.01, Top: [{'idx': 47, 'label': 'Atom47', 'confidence': '0.44', 'neighbors': 1}, {'idx': 31, 'label': 'Atom31', 'confidence': '0.44', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.50', 'neighbors': 0}]
Step 2: S=0.0712, κ=1.01, Top: [{'idx': 379, 'label': 'Atom379', 'confidence': '0.49', 'neighbors': 1}, {'idx': 107, 'label': 'Atom107', 'confidence': '0.43', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.49', 'neighbors': 0}]
Step 3: S=0.0752, κ=1.01, Top: [{'idx': 147, 'label': 'Atom147', 'confidence': '0.44', 'neighbors': 1}, {'idx': 21, 'label': 'Atom21', 'confidence': '0.43', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.49', 'neighbors': 0}]

Top Beliefs:
- Atom147 (conf: 0.44, neighbors: 1)
- Atom21 (conf: 0.43, neighbors: 1)
- Atom383 (conf: 0.49, neighbors: 0)
Last S: 0.0752
Edges: 2
Stable Symbols: 1
State saved (d=384). Stable symbols: 1
d=384, Active s

In [ ]:
test_feed(agent, "apple baked pie dessert not vegetable")  # Tips toward fruit, prunes veg
agent.get_status()


--- Feeding: 'apple baked pie dessert not vegetable' ---
Step 1: S=0.0706, κ=1.01, Top: [{'idx': 283, 'label': 'Atom283', 'confidence': '0.47', 'neighbors': 1}, {'idx': 62, 'label': 'Atom62', 'confidence': '0.43', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.48', 'neighbors': 0}]
Step 2: S=0.0741, κ=1.01, Top: [{'idx': 109, 'label': 'Atom109', 'confidence': '0.42', 'neighbors': 1}, {'idx': 75, 'label': 'Atom75', 'confidence': '0.42', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.48', 'neighbors': 0}]
Step 3: S=0.0720, κ=1.01, Top: [{'idx': 378, 'label': 'Atom378', 'confidence': '0.47', 'neighbors': 1}, {'idx': 189, 'label': 'Atom189', 'confidence': '0.43', 'neighbors': 1}, {'idx': 383, 'label': 'Atom383', 'confidence': '0.47', 'neighbors': 0}]

Top Beliefs:
- Atom378 (conf: 0.47, neighbors: 1)
- Atom189 (conf: 0.43, neighbors: 1)
- Atom383 (conf: 0.47, neighbors: 0)
Last S: 0.0720
Edges: 2
Stable Symbols: 1
State saved (d=384). Stable symbols